In [ ]:
##Hw 7 - Extra Credit for ECGR 4105
#Importing some libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import time


In [ ]:
#Importing the GPU and code is ran to check if gpu is available and what GPU

gpu_available = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_available}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Is the GPU available? True
Device cuda


In [ ]:
#Calculate the CPU and GPU timing as we can guess that the GPU will be quicker
x = torch.randn(5000, 5000)
startTime = time.time()
_ = torch.matmul(x, x)
endTime = time.time()
print(f"CPU time: {(endTime - startTime):6.5f}s")
x = x.to(device)
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
_ = torch.matmul(x, x)
end.record()
torch.cuda.synchronize()
print(f"GPU time: {0.001 * start.elapsed_time(end):6.5f}s")

CPU time: 2.99440s
GPU time: 0.08839s


In [ ]:


# ResNetBlock
class MyResNetBlock(nn.Module):
    def __init__(self, inChannels, outChannels, stride=1):
        super(MyResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(inChannels, outChannels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(outChannels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(outChannels, outChannels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(outChannels)

        # Adding a shortcut connection for residual learning
        self.shortcut = nn.Sequential()
        if stride != 1 or inChannels != outChannels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inChannels, outChannels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outChannels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)  # Adding the shortcut connection
        out = self.relu(out)
        return out

#  ResNet10
class MyResNet10(nn.Module):
    def __init__(self, block, numBlocks, numClasses=10):
        super(MyResNet10, self).__init__()
        self.inChannels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, numBlocks[0], stride=1)
        self.layer2 = self.make_layer(block, 32, numBlocks[1], stride=2)
        self.layer3 = self.make_layer(block, 64, numBlocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, numClasses)

    def make_layer(self, block, outChannels, numBlocks, stride):
        strides = [stride] + [1] * (numBlocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inChannels, outChannels, stride))
            self.inChannels = outChannels
        return nn.Sequential(*layers)
##Forward function
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

##Loading the dataset to train and loading the loader CIFAR10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# Training Function
def my_train(model, criterion, optimizer, epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

# Testing Function
def my_test(model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy on test set: {accuracy * 100:.2f}%")

# Example of usage with the customized ResNet10
my_resnet10 = MyResNet10(MyResNetBlock, [1, 1, 1])
my_criterion = nn.CrossEntropyLoss()
my_optimizer = optim.SGD(my_resnet10.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

my_train(my_resnet10, my_criterion, my_optimizer, epochs=300)
my_test(my_resnet10)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 1.6338231850158222
Epoch 2/300, Loss: 1.2637782560285096
Epoch 3/300, Loss: 1.090669776853698
Epoch 4/300, Loss: 0.9955482287022769
Epoch 5/300, Loss: 0.92734837684485
Epoch 6/300, Loss: 0.8703200556814213
Epoch 7/300, Loss: 0.8230771172382033
Epoch 8/300, Loss: 0.7896110429940626
Epoch 9/300, Loss: 0.7544280391977266
Epoch 10/300, Loss: 0.7251249007556749
Epoch 11/300, Loss: 0.6966854714219223
Epoch 12/300, Loss: 0.6648895406662045
Epoch 13/300, Loss: 0.6498498562961588
Epoch 14/300, Loss: 0.6317555347595678
Epoch 15/300, Loss: 0.6193975918661908
Epoch 16/300, Loss: 0.6020895501460566
Epoch 17/300, Loss: 0.5914106022214036
Epoch 18/300, Loss: 0.5740104504405995
Epoch 19/300, Loss: 0.5614126355523039
Epoch 20/300, Loss: 0.5515288040232476
Epoch 21/300, Loss: 0.5425750805288935
Epoch 22/300, Loss: 0.5373172192927211
Epoch 23/300, Loss: 0.5268640904246694
Epoch 24/300, Loss: 0.5230823021448786
